In [1]:
class BColours:
    BLUE = '\033[94m'
    CYAN = '\033[36m'
    GREEN = '\033[32m'
    MAGENTA = '\033[35m'
    RED = '\033[31m'
    ENDC = '\033[0m'

In [2]:
from bayes_opt import BayesianOptimization
import numpy as np
from datetime import datetime

In [10]:
def target(x, a_very_long_name):
    y = a_very_long_name
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/2 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    return 3*(2*a + b - c) - 1

In [64]:
class PrintLog(object):

    def __init__(self, params):

        self.ymax = None
        self.xmax = None
        self.params = params
        self.ite = 1

        self.start_time = datetime.now()
        self.last_round = datetime.now()

        # sizes of parameters name and all
        self.sizes = [max(len(ps), 6) for ps in params]

        # Sorted indexes to access parameters
        self.sorti = sorted(range(len(self.params)),
                            key=self.params.__getitem__)

    def print_header(self, initialization=True):

        if initialization:
            print("{}Initialization{}".format(BColours.RED, BColours.ENDC))
        else:
            print("{}Bayesian Optimization{}".format(BColours.RED, BColours.ENDC))

        print("-" * (28 + sum([s + 5 for s in self.sizes])) )

        print("{0:>{1}}".format("Step", 5), end=" | ")
        print("{0:>{1}}".format("Time", 6), end=" | ")
        print("{0:>{1}}".format("Value", 9), end=" | ")

        for index in self.sorti:
            print("{0:>{1}}".format(self.params[index], self.sizes[index] + 2), end=" | ")
        print('')

    def print_step(self, x, y):

        print("{:>5d}".format(self.ite), end=" | ")

        m, s = divmod((datetime.now() - self.last_round).total_seconds(), 60)
        print("{:>02d}m{:>02d}s".format(int(m), int(s)), end=" | ")

        if self.ymax is None or self.ymax < y:
            self.ymax = y
            self.xmax = x
            print("{0}{2: >9.5f}{1}".format(BColours.MAGENTA, BColours.ENDC, y), end=" | ")

            for index in self.sorti:
                print("{0}{2: >{3}.{4}f}{1}".format(BColours.GREEN, BColours.ENDC,
                                                    x[index], 
                                                    self.sizes[index] + 2,
                                                    min(self.sizes[index] - 3, 6 - 3)),
                      end=" | ")
        else:
            print("{: >9.5f}".format(y*10), end=" | ")
            for index in self.sorti:
                print("{0: >{1}.{2}f}".format(x[index], 
                                              self.sizes[index] + 2,
                                              min(self.sizes[index] - 3, 6 - 3)),
                      end=" | ")

        print()
        
        self.last_round = datetime.now()
        self.ite += 1

    def print_summary(self):
        pass
    
bo = BayesianOptimization(target, {'x': (-6, 6), 'a_very_long_name': (-6, 6)})
bo.plog = PrintLog(bo.keys)

In [65]:
gp_params = {'corr': 'absolute_exponential', 'nugget': 1e-15}

bo.explore({'x': [1, 3], 'a_very_long_name': [2, 3]})
# bo.initialize({-2: {'x': 1, 'y': 0}, -1.251: {'x': 1, 'y': 1.5}})
bo.maximize(init_points=3, n_iter=2, acq='ucb', kappa=3.29, **gp_params)

Initialization
------------------------------------------------------------
 Step |   Time |     Value |   a_very_long_name |        x | 
    1 | 00m00s |  -0.71120 |              2.000 |    1.000 | 
    2 | 00m00s |   0.27682 |              3.000 |    3.000 | 
    3 | 00m00s |  -9.99983 |             -2.722 |    5.666 | 
    4 | 00m00s |  -7.78257 |              0.450 |    1.947 | 
    5 | 00m00s |   1.40357 |              2.807 |    1.984 | 
Bayesian Optimization
------------------------------------------------------------
 Step |   Time |     Value |   a_very_long_name |        x | 
    6 | 00m08s | -10.00000 |              6.000 |   -6.000 | 
    7 | 00m04s | -10.00000 |             -6.000 |   -4.647 | 


In [21]:
'{:3.2f}'.format(25963.03764)

'25963.04'